In [1]:
import pandas as pd
import numpy as np

## Carga dos dados de entrada

In [2]:
# definir arquivo CSV de entrada
#arquivo = 'dados/afastamentos-nomes-sexo.csv'
arquivo = 'dados/cadastro-nomes-sexo.csv'
#arquivo = 'extra/nome_sexo_pf.csv.bz2'

In [3]:
partes = arquivo.split('.')
prefixo_arquivo = partes[0]
#sufixo_arquivo = '.'.join(partes[1:])
novo_arquivo = prefixo_arquivo + '-preparado'
novo_arquivo

'dados/cadastro-nomes-sexo-preparado'

In [4]:
df = pd.read_csv(arquivo, index_col=0)
df.head()

,SEXO,QTDE
PNOME,,
MARIA,F,20389
JOSE,M,15925
ANA,F,11497
JOAO,M,7515
CARLOS,M,7370


In [5]:
# remover valores nulos
df = df.loc[~df.index.isna()]
df.drop(df[df.index.isnull()].index, inplace=True)

In [6]:
total = df[df.index.isnull()].shape[0]
print("linhas nulas:", total)

linhas nulas: 0


In [7]:
# renomear índice e coluna
df.index.rename('NOME', inplace=True)
df.rename(columns={
    df.columns[0]: 'SEXO',
    df.columns[1]: 'QTDE'
}, inplace=True)

In [8]:
df.head()

,SEXO,QTDE
NOME,,
MARIA,F,20389
JOSE,M,15925
ANA,F,11497
JOAO,M,7515
CARLOS,M,7370


## Transformações nos dados

In [9]:
df['SEXO'] = df['SEXO'].map({'F': 0, 'M': 1, 'X': 9}).astype('uint8')

In [10]:
# remover sexo diferente de 0 ou 1
df.drop(df[df['SEXO'] > 1].index, inplace=True)

In [11]:
# converter para tipo de inteiro menor
df['QTDE'] = df['QTDE'].astype('uint32')
#np.iinfo(np.dtype('uint32'))

In [12]:
# criar reverso do nome
df['REV'] = df.index
df['REV'] = df['REV'].apply(lambda x: x[::-1])

In [13]:
df.head()

,SEXO,QTDE,REV
NOME,,,
MARIA,0,20389,AIRAM
JOSE,1,15925,ESOJ
ANA,0,11497,ANA
JOAO,1,7515,OAOJ
CARLOS,1,7370,SOLRAC


In [14]:
MAIOR_QTDE_LETRAS = 16 # fixado para o maior arquivo

for i in range(MAIOR_QTDE_LETRAS):
    df['L' + str(i)] = df['REV'].apply(
        lambda x: ord(x[i]) - 64 if len(x) > i else 0
    ).astype('uint8')

In [15]:
# remover coluna reversa
df.drop(['REV'], axis=1, inplace=True)

In [16]:
df.dtypes

SEXO     uint8
QTDE    uint32
L0       uint8
L1       uint8
L2       uint8
L3       uint8
L4       uint8
L5       uint8
L6       uint8
L7       uint8
L8       uint8
L9       uint8
L10      uint8
L11      uint8
L12      uint8
L13      uint8
L14      uint8
L15      uint8
dtype: object

## Análise dos resultados

### Amostra dos dados

In [17]:
df.head(10)

,SEXO,QTDE,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
NOME,,,,,,,,,,,,,,,,,,
MARIA,0,20389,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
JOSE,1,15925,5,19,15,10,0,0,0,0,0,0,0,0,0,0,0,0
ANA,0,11497,1,14,1,0,0,0,0,0,0,0,0,0,0,0,0,0
JOAO,1,7515,15,1,15,10,0,0,0,0,0,0,0,0,0,0,0,0
CARLOS,1,7370,19,15,12,18,1,3,0,0,0,0,0,0,0,0,0,0
PAULO,1,7218,15,12,21,1,16,0,0,0,0,0,0,0,0,0,0,0
ANTONIO,1,6982,15,9,14,15,20,14,1,0,0,0,0,0,0,0,0,0
LUIZ,1,6139,26,9,21,12,0,0,0,0,0,0,0,0,0,0,0,0
FRANCISCO,1,5795,15,3,19,9,3,14,1,18,6,0,0,0,0,0,0,0


In [18]:
print("total de linhas:", df.shape[0])

total de linhas: 41146


### Quantidades por sexo

In [19]:
df2 = df[['SEXO', 'L0']].groupby(by=['SEXO']).count().rename({'L0': 'CONT'}, axis=1)
total = df2['CONT'].sum()
df2['PERC'] = df2['CONT'].apply(lambda x: int(x / total * 10000) / 100)
df2

,CONT,PERC
SEXO,,
0,21890,53.20
1,19256,46.79


## Gravação do arquivo final

In [20]:
df.to_csv(novo_arquivo + '.csv.bz2', compression='bz2')

In [21]:
df.to_pickle(novo_arquivo + '.pkl.bz2')